In [1]:
import gymnasium as gym
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import random

# Define the PGN Network
class PGN(nn.Module):
    def __init__(self):
        super(PGN, self).__init__()
        self.fc1 = nn.Linear(4, 24)  # CartPole state is 4-dimensional
        self.fc2 = nn.Linear(24, 24)
        self.fc3 = nn.Linear(24, 2)  # Two actions: push cart left or right

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = torch.softmax(self.fc3(x), dim=1) # Need softmax for PG to output probabilities 
        
        return x

In [6]:
# Initialize environment, model, and optimizer
env = gym.make('CartPole-v1')
model = PGN()
learning_rate = 0.0001
optimizer = optim.Adam(model.parameters(), lr = learning_rate)

# Training parameters
num_episodes = 10000
gamma = 0.90 # Discount factor

for episode in range(num_episodes):
    state = env.reset()[0]
    
    last_episode = []
    rewards = [] 
    actions = []
    
    # Simulate a full episode
    for t in range(1, 10000):  
        state_tensor = torch.from_numpy(state).float().unsqueeze(0)
        last_episode.append(state_tensor)

        with torch.no_grad():
            # Select action from action probability dist. 
            action = torch.multinomial(model(state_tensor), 1).item()
            actions.append(action)
        
        next_state, reward, terminated, truncated, info = env.step(action)
        done = terminated or truncated
        
        if done:
            reward = -5
        
        state = next_state

        ############
        rewards.append(reward)
        ############

        if done:
            break
            
    # Train on past episode 
    rev_cumul_reward = 0
    episode_loss = 0
    for ridx, state in enumerate(reversed(last_episode)): # traverse list in reverse
        rev_cumul_reward *= gamma
        rev_cumul_reward += rewards[-ridx-1]

        # This line below is crucial need action to be same as one from step in episode (to boost or reduce its prob)
        # use target model so that action probs are same as at initial eval time! (nope just treat full episode as minibatch)
        # weird target model makes it much worse even with target_update = 1
        action_probability = model(last_episode[-ridx-1])[0][actions[-ridx-1]]  # should make this more readable

        log_policy = torch.log(action_probability) # Will always be negative 
        
        loss = -1 * log_policy * rev_cumul_reward
        episode_loss += loss

    episode_loss.backward()
    optimizer.step()
    episode_loss = 0

    if episode % 50 == 0:
        print(t)
        
        

env.close()

27
31
37
28
15
20
55
40
37
29
41
32
31
58


KeyboardInterrupt: 